In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [8]:
import torch
print("Pytorch version：")
print(torch.__version__)
print("CUDA Version: ")
print(torch.version.cuda)
print("cuDNN version is :")
print(torch.backends.cudnn.version())

Pytorch version：
2.5.0+cu124
CUDA Version: 
12.4
cuDNN version is :
90100


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

/projectnb/cs505aw/students/ziyechen/.conda/envs/myenv310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import csv

# Classifier

In [9]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_Classifier", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 10240,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.10.0: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L40S. Max memory: 44.418 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.10.0 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [47]:
evaluation_prompt = """As a mathematical assistant, You need to analyze the problem to find out what type of problem it belongs to in Real Analysis. Provide the Problem_Type and the Knowledges which may be used to solve this problem.

### Problem:
{}

### Problem_Type:
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    Problem       = examples["Problem"]
    texts = []
    for problem in Problem:
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = evaluation_prompt.format(problem) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv',data_files = './benchmark_data.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 39/39 [00:00<00:00, 1006.98 examples/s]


In [16]:
dataset[:2]

{'Number': ['1_4', '1_6'],
 'ProblemType': ['Sequences and Limits', 'Sequences and Limits'],
 'Problem': ['Suppose that $a_n$ and $b_n$ converge to $\\alpha$ and $\\beta$ as $n \\to \\infty$ respectively. Show that the sequence\n$$\n\\frac{a_0 b_n + a_1 b_{n-1} + \\cdots + a_n b_0}{n}\n$$\nconverges to $\\alpha \\beta$ as $n \\to \\infty$.',
  "For any positive sequence $\\{a_n\\}_{n \\geq 1}$, show that\n$$\n\\left( \\frac{a_1 + a_{n+1}}{a_n} \\right)^n > e\n$$\nfor infinitely many $n$'s, where $e$ is the base of the natural logarithm. Prove moreover that the constant $e$ on the right-hand side cannot in general be replaced by any larger number."],
 'Solution': ['Let $M$ be an upper bound of the two convergent sequences $|a_n|$ and $|b_n|$. For any $\\epsilon > 0$ we can take a positive integer $N$ satisfying $|a_n - \\alpha| < \\epsilon$ and $|b_n - \\beta| < \\epsilon$ for all integers $n$ greater than $N$. If $n$ is greater than $N^2$, then\n$$\n|a_k b_{n-k} - \\alpha \\beta| \\leq

In [50]:
results = []
TYPE = []
KNOWLEDGE = []
for row in dataset:
    number = row['Number']
    problem = row['Problem']
    
    # Format the prompt
    prompt = evaluation_prompt.format(problem)
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    
    # Generate the solution
    output_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        # streamer = text_streamer,
        max_new_tokens=4096,
        pad_token_id=tokenizer.eos_token_id,
        # do_sample=True,   # Enable sampling for more diverse outputs
        # temperature=0.7   # Adjust temperature for creativity
    )
    
    # Decode the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Extract the generated solution
    problem_type = output_text[output_text.find('### Problem_Type')+17:output_text.find('### Knowledge')].strip()
    knowledge = output_text[output_text.find('### Knowledge')+14:].strip()
    
    # Append the result
    results.append({
        'Number': number,
        'Problem': problem,
        'Problem_Type': problem_type,
        'Knowledge': knowledge
    })

    TYPE.append(problem_type)
    KNOWLEDGE.append(knowledge)


In [56]:
print(len(dataset)==len(TYPE))
print(len(dataset)==len(KNOWLEDGE))

True
True


# Problem Solver

In [57]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_ProblemSolver", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 10240,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.10.0: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L40S. Max memory: 44.418 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: unsloth/Qwen2.5-Math-7B-bnb-4bit can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 2.5, it can be magically be extended to 10240!


In [ ]:
alpaca_prompt = """As a mathematical assistant, solve the following problem. Provide a detailed, step-by-step solution using rigorous mathematical reasoning. If the problem requires the use of the $\epsilon$-$\delta$ method (e.g., when proving limits or continuity), ensure that you apply it appropriately. Use precise mathematical language and notation throughout your solution.

### Problem_Type:
{}

### Problem:
{}

### Knowledge:
{}

### Solution:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    Problem_Type    = examples["ProblemType"]
    Problem       = examples["Problem"]
    Knowledge = examples["Knowledge"]
    Solution  = examples["Solution"]
    texts = []
    for problem_type, problem, knowledge, solution in zip(Problem_Type, Problem, Knowledge, Solution):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(problem_type, problem, knowledge, solution) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv',data_files = './benchmark_data_with_knowledge.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [58]:
alpaca_prompt = """As a mathematical assistant, solve the following problem. Provide a detailed, step-by-step solution using rigorous mathematical reasoning. If the problem requires the use of the $\epsilon$-$\delta$ method (e.g., when proving limits or continuity), ensure that you apply it appropriately. Use precise mathematical language and notation throughout your solution.

### Problem_Type:
{}

### Problem:
{}

### Knowledge:
{}

### Solution:
{}"""

In [59]:
from datasets import load_dataset
dataset = load_dataset('csv',data_files = './benchmark_data.csv', split='train')

In [61]:
results = []
N_ROW = 0
for row in dataset:
    number = row['Number']
    problem_type = TYPE[N_ROW]
    problem = row['Problem']
    knowledge = KNOWLEDGE[N_ROW]
    
    # Format the prompt
    prompt = alpaca_prompt.format(problem_type, problem, knowledge, "")
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    
    # Generate the solution
    output_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        # streamer = text_streamer,
        max_new_tokens=2048,
        pad_token_id=tokenizer.eos_token_id,
        # do_sample=True,   # Enable sampling for more diverse outputs
        # temperature=0.7   # Adjust temperature for creativity
    )
    
    # Decode the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Extract the generated solution
    generated_solution = output_text[len(prompt):].strip()
    
    # Append the result
    results.append({
        'Number': number,
        'Problem': problem,
        'Solution': generated_solution
    })

    N_ROW += 1



In [62]:
results[1]

{'Number': '1_6',
 'Problem': "For any positive sequence $\\{a_n\\}_{n \\geq 1}$, show that\n$$\n\\left( \\frac{a_1 + a_{n+1}}{a_n} \\right)^n > e\n$$\nfor infinitely many $n$'s, where $e$ is the base of the natural logarithm. Prove moreover that the constant $e$ on the right-hand side cannot in general be replaced by any larger number.",
 'Solution': "Let $M$ be any arbitrary positive number. It suffices to show that the inequality\n\\[\n\\frac{1}{n} \\log \\frac{a_1 + a_{n+1}}{a_n} \\geq \\frac{M}{n}\n\\tag{3.1}\n\\]\nholds for all sufficiently large $n$. We will prove (3.1) by Cauchy's method. Let $p$ and $q$ be any positive integers; then we have\n\\[\n\\frac{1}{q-p} \\sum_{n=p}^{q-1} \\frac{1}{n} \\log \\frac{a_1 + a_{n+1}}{a_n} \\geq \\frac{1}{p}.\n\\]\n\\]\nNow let $p$ be a fixed positive integer and $q$ vary. Since the right-hand side converges to $0$ as $q \\to \\infty$, it follows that the superior limit of of the left-hand side as $q \\to \\infty$ is $\\geq 0$. Therefore the

In [63]:
# Step 4: Save the Results to a New CSV File
with open('solutions.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Number','Problem', 'Solution']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print("Solutions have been generated and saved to 'solutions.csv'.")

Solutions have been generated and saved to 'solutions.csv'.
